# Budowa modelu regresji: przygotowanie i wizualizacja danych

## **Regresja liniowa dla dyń - Lekcja 2**
#### Wprowadzenie

Teraz, gdy masz już narzędzia potrzebne do rozpoczęcia budowy modeli uczenia maszynowego za pomocą Tidymodels i Tidyverse, jesteś gotowy, aby zacząć zadawać pytania dotyczące swoich danych. Pracując z danymi i stosując rozwiązania ML, bardzo ważne jest, aby umieć zadawać właściwe pytania, które pozwolą w pełni wykorzystać potencjał Twojego zbioru danych.

W tej lekcji dowiesz się:

-   Jak przygotować dane do budowy modelu.

-   Jak używać `ggplot2` do wizualizacji danych.

Pytanie, na które chcesz uzyskać odpowiedź, określi, jaki typ algorytmów ML zastosujesz. Jakość odpowiedzi, którą otrzymasz, będzie w dużej mierze zależała od charakteru Twoich danych.

Przyjrzyjmy się temu, wykonując praktyczne ćwiczenie.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Ilustracja autorstwa @allison_horst</figcaption>


<!--![Ilustracja autorstwa \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.pl.jpg)<br>Ilustracja autorstwa \@allison_horst-->


## 1. Importowanie danych o dyniach i przywoływanie Tidyverse

Do przeprowadzenia tej lekcji będziemy potrzebować następujących pakietów:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) to [zbiór pakietów R](https://www.tidyverse.org/packages) zaprojektowany, aby uczynić analizę danych szybszą, łatwiejszą i bardziej przyjemną!

Możesz je zainstalować za pomocą:

`install.packages(c("tidyverse"))`

Poniższy skrypt sprawdza, czy masz zainstalowane pakiety potrzebne do ukończenia tego modułu, i instaluje je, jeśli czegoś brakuje.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Teraz uruchommy kilka pakietów i załadujmy [dane](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) udostępnione na potrzeby tej lekcji!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Szybkie użycie `glimpse()` natychmiast pokazuje, że są puste wartości oraz mieszanka ciągów znaków (`chr`) i danych numerycznych (`dbl`). Kolumna `Date` jest typu znakowego, a także znajduje się dziwna kolumna o nazwie `Package`, gdzie dane to mieszanka wartości takich jak `sacks`, `bins` i innych. Dane, w rzeczywistości, są trochę chaotyczne 😤.

W rzeczywistości, rzadko zdarza się otrzymać zestaw danych, który jest całkowicie gotowy do użycia w celu stworzenia modelu ML od razu. Ale nie martw się, w tej lekcji nauczysz się, jak przygotować surowy zestaw danych, korzystając ze standardowych bibliotek R 🧑‍🔧. Nauczysz się również różnych technik wizualizacji danych.📈📊
<br>

> Przypomnienie: Operator pipe (`%>%`) wykonuje operacje w logicznej kolejności, przekazując obiekt dalej do funkcji lub wyrażenia. Możesz myśleć o operatorze pipe jako o powiedzeniu "a następnie" w swoim kodzie.


## 2. Sprawdź brakujące dane

Jednym z najczęstszych problemów, z jakimi muszą się zmierzyć analitycy danych, są niekompletne lub brakujące dane. W R brakujące lub nieznane wartości są reprezentowane za pomocą specjalnej wartości: `NA` (Not Available).

Jak więc możemy sprawdzić, czy ramka danych zawiera brakujące wartości?  
<br>
- Jednym z prostych sposobów jest użycie funkcji bazowej R `anyNA`, która zwraca logiczne wartości `TRUE` lub `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Świetnie, wygląda na to, że brakuje pewnych danych! To dobry punkt wyjścia.

-   Innym sposobem byłoby użycie funkcji `is.na()`, która wskazuje, które poszczególne elementy kolumny są brakujące, zwracając wartość logiczną `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Ok, zadanie wykonane, ale przy tak dużej ramce danych jak ta, byłoby to nieefektywne i praktycznie niemożliwe, aby przejrzeć wszystkie wiersze i kolumny indywidualnie😴.

-   Bardziej intuicyjnym sposobem byłoby obliczenie sumy brakujących wartości dla każdej kolumny:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Dużo lepiej! Brakuje danych, ale może nie będzie to miało znaczenia dla realizacji zadania. Zobaczmy, jakie wnioski przyniesie dalsza analiza.

> Oprócz świetnych zestawów pakietów i funkcji, R posiada bardzo dobrą dokumentację. Na przykład, użyj `help(colSums)` lub `?colSums`, aby dowiedzieć się więcej o tej funkcji.


## 3. Dplyr: Gramatyka manipulacji danymi

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Grafika autorstwa @allison_horst</figcaption>


<!--![Grafika autorstwa \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.pl.png)<br/>Grafika autorstwa \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), pakiet w Tidyverse, to gramatyka manipulacji danymi, która oferuje spójny zestaw czasowników pomagających rozwiązywać najczęstsze wyzwania związane z manipulacją danymi. W tej sekcji przyjrzymy się niektórym czasownikom dplyr!


#### dplyr::select()

`select()` to funkcja w pakiecie `dplyr`, która pozwala wybrać kolumny do zachowania lub wykluczenia.

Aby ułatwić pracę z ramką danych, usuń kilka jej kolumn, używając `select()`, pozostawiając tylko te, które są potrzebne.

Na przykład w tym ćwiczeniu nasza analiza będzie dotyczyć kolumn `Package`, `Low Price`, `High Price` i `Date`. Wybierzmy te kolumny.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` to funkcja w pakiecie `dplyr`, która pozwala tworzyć lub modyfikować kolumny, zachowując istniejące kolumny.

Ogólna struktura `mutate` wygląda następująco:

`data %>%   mutate(new_column_name = what_it_contains)`

Przyjrzyjmy się działaniu `mutate`, używając kolumny `Date`, wykonując następujące operacje:

1.  Konwersja dat (obecnie typu znakowego) na format miesiąca (są to daty w formacie amerykańskim, czyli `MM/DD/YYYY`).

2.  Wyodrębnienie miesiąca z dat do nowej kolumny.

W R, pakiet [lubridate](https://lubridate.tidyverse.org/) ułatwia pracę z danymi typu Date-time. Zatem użyjmy `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` i zobaczmy, jak osiągnąć powyższe cele. Możemy usunąć kolumnę `Date`, ponieważ nie będzie nam już potrzebna w kolejnych operacjach.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Hurra! 🤩

Teraz utwórzmy nową kolumnę `Price`, która będzie reprezentować średnią cenę dyni. Następnie obliczmy średnią z kolumn `Low Price` i `High Price`, aby wypełnić nową kolumnę Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Tak!💪

„Ale chwileczkę!”, powiesz po szybkim przejrzeniu całego zestawu danych za pomocą `View(pumpkins)`, „Coś tu jest dziwnego!”🤔

Jeśli spojrzysz na kolumnę `Package`, dynie są sprzedawane w różnych konfiguracjach. Niektóre są sprzedawane w miarach `1 1/9 bushel`, inne w miarach `1/2 bushel`, niektóre na sztuki, niektóre na wagę, a jeszcze inne w dużych pudełkach o różnych szerokościach.

Sprawdźmy to:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Niesamowite!👏

Dynie wydają się być bardzo trudne do ważenia w sposób spójny, więc przefiltrujmy je, wybierając tylko te dynie, które mają ciąg znaków *bushel* w kolumnie `Package` i umieśćmy je w nowej ramce danych `new_pumpkins`.


#### dplyr::filter() i stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): tworzy podzbiór danych zawierający tylko **wiersze**, które spełniają określone warunki, w tym przypadku dynie z ciągiem *bushel* w kolumnie `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): wykrywa obecność lub brak wzorca w ciągu znaków.

Pakiet [`stringr`](https://github.com/tidyverse/stringr) dostarcza proste funkcje do typowych operacji na ciągach znaków.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Możesz zobaczyć, że ograniczyliśmy się do około 415 wierszy danych zawierających dynie na korzec.🤩


#### dplyr::case_when()

**Ale chwileczkę! Jest jeszcze jedna rzecz do zrobienia**

Zauważyłeś, że ilość buszli różni się w zależności od wiersza? Musisz znormalizować ceny, aby pokazać cenę za buszel, a nie za 1 1/9 czy 1/2 buszla. Czas na trochę matematyki, aby to ustandaryzować.

Użyjemy funkcji [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html), aby *zmodyfikować* kolumnę Price w zależności od pewnych warunków. `case_when` pozwala na wektoryzację wielu instrukcji `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Teraz możemy przeanalizować cenę za jednostkę na podstawie ich miary w buszlach. Całe to badanie buszli dyni pokazuje, jak `ważne` jest `zrozumienie charakteru swoich danych`!

> ✅ Według [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), waga buszla zależy od rodzaju produktu, ponieważ jest to miara objętości. "Buszel pomidorów, na przykład, powinien ważyć 56 funtów... Liście i zielenina zajmują więcej miejsca przy mniejszej wadze, więc buszel szpinaku waży tylko 20 funtów." To wszystko jest dość skomplikowane! Nie zawracajmy sobie głowy przeliczaniem buszli na funty, zamiast tego ustalmy cenę za buszel. Całe to badanie buszli dyni pokazuje, jak bardzo ważne jest zrozumienie charakteru swoich danych!
>
> ✅ Zauważyłeś, że dynie sprzedawane na pół buszla są bardzo drogie? Czy potrafisz wyjaśnić dlaczego? Podpowiedź: małe dynie są znacznie droższe niż duże, prawdopodobnie dlatego, że jest ich znacznie więcej w buszlu, biorąc pod uwagę niewykorzystaną przestrzeń zajmowaną przez jedną dużą pustą dynię na ciasto.


Na koniec, dla czystej przyjemności przygody 💁‍♀️, przenieśmy kolumnę Miesiąc na pierwszą pozycję, czyli `przed` kolumnę `Pakiet`.

Do zmiany pozycji kolumn używamy `dplyr::relocate()`.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Dobra robota!👌 Masz teraz czysty, uporządkowany zestaw danych, na którym możesz zbudować swój nowy model regresji!
<br>


## 4. Wizualizacja danych za pomocą ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografika autorstwa Dasani Madipalli</figcaption>


<!--![Infografika autorstwa Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.pl.png){width="600"}-->

Istnieje *mądre* powiedzenie, które brzmi:

> "Prosty wykres dostarczył analitykowi danych więcej informacji niż jakiekolwiek inne narzędzie." --- John Tukey

Częścią pracy naukowca danych jest przedstawienie jakości i charakteru danych, z którymi pracuje. Aby to zrobić, często tworzy interesujące wizualizacje, takie jak wykresy, grafy i diagramy, które pokazują różne aspekty danych. Dzięki temu mogą wizualnie ukazać zależności i luki, które w innym przypadku byłyby trudne do zauważenia.

Wizualizacje mogą również pomóc w określeniu, która technika uczenia maszynowego będzie najbardziej odpowiednia dla danych. Na przykład wykres punktowy, który wydaje się podążać za linią, wskazuje, że dane mogą być dobrym kandydatem do zastosowania regresji liniowej.

R oferuje wiele systemów do tworzenia wykresów, ale [`ggplot2`](https://ggplot2.tidyverse.org/index.html) jest jednym z najbardziej eleganckich i wszechstronnych. `ggplot2` pozwala na tworzenie wykresów poprzez **łączenie niezależnych komponentów**.

Zacznijmy od prostego wykresu punktowego dla kolumn Price i Month.

W tym przypadku zaczniemy od [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), dostarczymy zbiór danych i mapowanie estetyczne (za pomocą [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), a następnie dodamy warstwy (takie jak [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) dla wykresów punktowych.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Czy ten wykres jest przydatny 🤷? Czy coś w nim Cię zaskakuje?

Nie jest szczególnie użyteczny, ponieważ jedyne, co robi, to przedstawia dane jako rozrzut punktów w danym miesiącu.
<br>


### **Jak uczynić to przydatnym?**

Aby wykresy przedstawiały użyteczne dane, zazwyczaj trzeba jakoś pogrupować dane. Na przykład w naszym przypadku, obliczenie średniej ceny dyni dla każdego miesiąca dostarczyłoby więcej informacji o ukrytych wzorcach w naszych danych. To prowadzi nas do kolejnego szybkiego przeglądu funkcji **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Grupowanie i agregacja w R mogą być łatwo wykonane za pomocą

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` zmienia jednostkę analizy z całego zbioru danych na poszczególne grupy, takie jak miesiące.

-   `dplyr::summarize()` tworzy nową ramkę danych z jedną kolumną dla każdej zmiennej grupującej oraz jedną kolumną dla każdej statystyki podsumowującej, którą określisz.

Na przykład, możemy użyć `dplyr::group_by() %>% summarize()` do pogrupowania dyni według kolumny **Month**, a następnie obliczyć **średnią cenę** dla każdego miesiąca.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Zwięźle!✨

Kategoryczne cechy, takie jak miesiące, lepiej przedstawiać za pomocą wykresu słupkowego 📊. Warstwy odpowiedzialne za wykresy słupkowe to `geom_bar()` i `geom_col()`. Sprawdź `?geom_bar`, aby dowiedzieć się więcej.

Zróbmy jeden!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 To bardziej przydatna wizualizacja danych! Wygląda na to, że najwyższe ceny dyni występują we wrześniu i październiku. Czy to odpowiada Twoim oczekiwaniom? Dlaczego tak lub dlaczego nie?

Gratulacje z ukończenia drugiej lekcji 👏! Przygotowałeś dane do budowy modelu, a następnie odkryłeś więcej informacji dzięki wizualizacjom!



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
